In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
# Set for less chatty log messages
import logging

logger = logging.getLogger('ATOM')
logger.setLevel(logging.INFO)
# ignore sklearn future warnings
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from atomsci.ddm.pipeline import predict_from_model as pfm
from sklearn.metrics import r2_score

2024-04-16 20:31:04.328923: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 20:31:04.422085: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 20:31:04.422139: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 20:31:04.422200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-16 20:31:04.443911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 20:31:04.444466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")

import pandas as pd
import os, json, sys, glob, pickle

from atomsci.ddm.pipeline import perf_data

# We temporarily disable warnings for demonstration.
# FutureWarnings and DeprecationWarnings are present from some of the AMPL 
# dependency modules.
import warnings
warnings.filterwarnings('ignore')



import pandas as pd
import os, json, sys, glob, pickle
import requests


from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.utils.struct_utils as struct_utils
from atomsci.ddm.pipeline import perf_plots as pp
import atomsci.ddm.pipeline.diversity_plots as dp

In [2]:
# excel file with the logP dataset
#logP = pd.DataFrame(pd.read_excel('dataset/Dataset_and_Predictions.xlsx')) 
logP = pd.read_csv('dataset/Dataset_and_Predictions.csv')
logP

,SMILES,CAS,ChemID,logP\nexperimental\n(corrected),logP\npredicted,DNNtaut,DNNmono,KOWWIN,OCHEM,ALOGPS,...,assignments,original SMILES,original SMILES\nwithout ions\n,original SMILES randomly selected (including tautomers),original SMILES most likely tautomer,SMILES randomly selected,SMILES randomly selected without ions,canonical SMILES,SMILES including explicit H atoms,classified as ion?
0,CCC1(NC(=O)N(C)C1=O)c1ccccc1,50-12-4,100008,1.69,NaN,2.04,2.08,1.65,1.61,1.64,...,NaN,test,test,test,none,none,none,test,none,no
1,[H]N1C(=O)N(C(=O)C1(C1=C([H])C([H])=C([H])C([H...,50-12-4,100008,1.69,NaN,1.97,1.16,1.65,1.61,1.64,...,NaN,none,none,none,none,none,none,none,test,no
2,CCC1(NC(=O)N(C)C1=O)C1=CC=CC=C1,50-12-4,100008,1.69,NaN,2.04,2.08,1.65,1.61,1.64,...,NaN,none,none,none,test,none,none,none,none,no
3,[H]OC1=NC(C(=O)N1C([H])([H])[H])(C1=C([H])C([H...,50-12-4,100008,1.69,NaN,1.71,1.26,1.05,1.25,1.43,...,NaN,none,none,none,none,test,test,none,none,no
4,CCC1(N=C(O)N(C)C1=O)C1=CC=CC=C1,50-12-4,100008,1.69,NaN,1.71,1.94,1.05,1.25,1.43,...,NaN,none,none,none,none,none,none,none,none,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12704,S=C(NN=C(c1ccccn1)c1ccccn1)Nc1ccccc1,67629-66-7,115633,1.96,NaN,2.82,3.13,4.24,2.76,3.42,...,NaN,test,test,none,none,none,none,none,none,no
12705,S=C(NN=C(c1ccccn1)c2ccccn2)Nc3ccccc3,67629-66-7,115633,1.96,NaN,2.82,3.13,4.24,2.76,3.42,...,NaN,none,none,none,none,none,none,none,none,no
12706,[H]SC(N=NC([H])(C1=C([H])C([H])=C([H])C([H])=N...,67629-66-7,115633,1.96,NaN,2.69,2.34,4.91,3.22,4.24,...,NaN,none,none,none,none,none,none,none,none,no
12707,[H]N(C(=S)N=NC([H])(C1=C([H])C([H])=C([H])C([H...,67629-66-7,115633,1.96,NaN,2.63,2.22,5.48,3.43,4.39,...,NaN,none,none,none,none,none,none,none,none,no


In [15]:
curated_logPclass = pd.read_csv("dataset/curated_logP.csv")
# high vs not high
curated_logPclass['logP_level'] = pd.cut(curated_logPclass['average_logP'], bins=[-float('inf'), 4, float('inf')],
                              labels=[0,1], right=False)

In [5]:
curated_logPclass.to_csv('dataset/curated_logPclass.csv', index=False)

In [4]:
curated_logPclass

,compound_id,base_rdkit_smiles,relation,average_logP,logP_level
0,CHEMBL102422,BrC=C(Br)Br,NaN,3.20,0
1,CHEMBL101049,Brc1ccc(Br)cc1,NaN,3.79,0
2,CHEMBL107341,Brc1cccc(Oc2ccc(Br)c(Br)c2Br)c1,NaN,6.05,1
3,CHEMBL103677,Brc1ccccc1-c1ccccc1,NaN,4.59,1
4,CHEMBL100362,C#CC,NaN,0.94,0
...,...,...,...,...,...
1384,CHEMBL101649,c1ccc2cc3c(cc2c1)-c1cccc2cccc-3c12,NaN,6.11,1
1385,CHEMBL101775,c1cnc2nccnc2c1,NaN,-0.04,0
1386,CHEMBL102028,c1cncc([C@@H]2CCCCN2)c1,NaN,0.97,0
1387,CHEMBL104361,c1coc(C2=Nc3ccccc3[N]2)c1,NaN,2.67,0


In [18]:
curated_logPclass.logP_level.value_counts()

logP_level
0    1218
1     171
Name: count, dtype: int64

In [ ]:

#dataset_key='dataset/curated_logP.csv'
dataset_key='dataset/curated_logPclass.csv'
model_dir='dataset'
descriptor_type = 'rdkit_raw'
best_model_dir = 'dataset/logP_models/best_models'
split_uuid = MP.split_dataset()


if not os.path.exists(f'./{best_model_dir}'):
    os.mkdir(f'./{best_model_dir}')
    
if not os.path.exists(f'./{model_dir}'):
    os.mkdir(f'./{model_dir}')

import pandas as pd
from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse

dataset_file = 'dataset/curated_logPclass.csv'
odir='dataset'

response_col = "logP_level"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"
#split_uuid = "c35aeaab-910c-4dcf-8f9f-04b55179aa1a"

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_uuid" : split_uuid,
        "split_only": "False",
        "featurizer": "computed_descriptors",
        "descriptor_type" : "rdkit_raw",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        "rerun": "False",
        "result_dir": odir
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

In [7]:
import pandas as pd
from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse

dataset_file = 'dataset/curated_logPclass.csv'
odir='dataset'

response_col = "logP_level"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"
#split_uuid = "c35aeaab-910c-4dcf-8f9f-04b55179aa1a"

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_strategy": "train_valid_test",
        "split_only": "False",
        "splitter": "fingerprint",
        "split_valid_frac": "0.10",
        "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": ""
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

In [8]:
!tar -tf *.tar.gz

./best_model/
./best_model/model.joblib
./model_metadata.json
./model_metrics.json


In [9]:
!tar xzf *.tar.gz -C /tmp

In [10]:
!cat /tmp/model_metadata.json | head

{
    "ecfp_specific": {
        "ecfp_radius": 2,
        "ecfp_size": 1024
    },
    "model_parameters": {
        "ampl_version": "1.6.0",
        "class_number": 2,
        "featurizer": "ecfp",
        "hyperparam_uuid": null,


In [12]:
import joblib
# load the model from disk
loaded_model = joblib.load("/tmp/best_model/model.joblib")
loaded_model

RandomForestClassifier(max_features=32, n_estimators=500, n_jobs=-1)

In [13]:
import json
import pandas as pd
data = json.load(open('/tmp/model_metadata.json'))
# pd.DataFrame(data["model_parameters"])
data['model_parameters']

{'ampl_version': '1.6.0',
 'class_number': 2,
 'featurizer': 'ecfp',
 'hyperparam_uuid': None,
 'model_bucket': 'public',
 'model_choice_score_type': 'roc_auc',
 'model_type': 'RF',
 'num_model_tasks': 1,
 'prediction_type': 'classification',
 'save_results': False,
 'system': 'twintron-blue',
 'time_generated': 1713290080.0563927,
 'transformer_bucket': 'public',
 'transformer_key': None,
 'transformer_oid': None,
 'transformers': True,
 'uncertainty': True}

In [14]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test

{'roc_auc_score': 0.7569230769230768,
 'prc_auc_score': 0.19608126224323408,
 'cross_entropy': 0.23520962410485496,
 'precision': 0.0,
 'recall_score': 0.0,
 'npv': 0.9285714285714286,
 'accuracy_score': 0.9285714285714286,
 'bal_accuracy': 0.5,
 'kappa': 0.0,
 'matthews_cc': 0.0,
 'confusion_matrix': [[[130, 0], [10, 0]]],
 'num_compounds': 140}

# LogP low classification

In [5]:
curated_logPclasslow = pd.read_csv("dataset/curated_logP.csv")
# high vs not high
curated_logPclasslow['logP_level'] = pd.cut(curated_logPclasslow['average_logP'], bins=[-float('inf'), 2, float('inf')],
                              labels=[1,0], right=False)

In [6]:
curated_logPclasslow.to_csv('dataset/curated_logPclasslow.csv', index=False)

In [8]:
import pandas as pd
from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse

dataset_file = 'dataset/curated_logPclasslow.csv'
odir='dataset'

response_col = "logP_level"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"
#split_uuid = "c35aeaab-910c-4dcf-8f9f-04b55179aa1a"

params = {
        "verbose": "True",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "classification",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_strategy": "train_valid_test",
        "split_only": "False",
        "splitter": "fingerprint",
        "split_valid_frac": "0.10",
        "split_test_frac": "0.10",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": ""
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

INFO:ATOM:Featurized dataset not previously saved for dataset curated_logPclasslow, creating new
INFO:ATOM:Featurizing sample 0
INFO:ATOM:Featurizing sample 1000
INFO:ATOM:Splitting data by fingerprint
INFO:ATOM:Dataset split table saved to /home/jupyter/AMPL/atomsci/logP/dataset/curated_logPclasslow_train_valid_test_fingerprint_42837689-e52c-4405-a0e1-2e8e5deaaa6b.csv
INFO:ATOM:Fitting random forest model
INFO:ATOM:Fold 0: training roc_auc_score = 1.000, validation roc_auc_score = 0.779, test roc_auc_score = 0.829
INFO:ATOM:Wrote model tarball to curated_logPclasslow_model_8dc2b45e-b762-4705-b412-8982c8d6311c.tar.gz


In [9]:
!tar -tf *.tar.gz

tar: curated_logPclasslow_model_8dc2b45e-b762-4705-b412-8982c8d6311c.tar.gz: Not found in archive
tar: Exiting with failure status due to previous errors


In [10]:
!tar xzf *.tar.gz -C /tmp

tar: curated_logPclasslow_model_8dc2b45e-b762-4705-b412-8982c8d6311c.tar.gz: Not found in archive
tar: Exiting with failure status due to previous errors


In [11]:
!cat /tmp/model_metadata.json | head

{
    "ecfp_specific": {
        "ecfp_radius": 2,
        "ecfp_size": 1024
    },
    "model_parameters": {
        "ampl_version": "1.6.0",
        "class_number": 2,
        "featurizer": "ecfp",
        "hyperparam_uuid": null,


In [12]:
import joblib
# load the model from disk
loaded_model = joblib.load("/tmp/best_model/model.joblib")
loaded_model

RandomForestClassifier(max_features=32, n_estimators=500, n_jobs=-1)

In [13]:
import json
import pandas as pd
data = json.load(open('/tmp/model_metadata.json'))
# pd.DataFrame(data["model_parameters"])
data['model_parameters']

{'ampl_version': '1.6.0',
 'class_number': 2,
 'featurizer': 'ecfp',
 'hyperparam_uuid': None,
 'model_bucket': 'public',
 'model_choice_score_type': 'roc_auc',
 'model_type': 'RF',
 'num_model_tasks': 1,
 'prediction_type': 'classification',
 'save_results': False,
 'system': 'twintron-blue',
 'time_generated': 1713290080.0563927,
 'transformer_bucket': 'public',
 'transformer_key': None,
 'transformer_oid': None,
 'transformers': True,
 'uncertainty': True}

In [14]:
pred_data_test = pl.model_wrapper.get_perf_data(subset="test", epoch_label="best")
pred_results_test = pred_data_test.get_prediction_results()
pred_results_test


{'roc_auc_score': 0.8285984848484849,
 'prc_auc_score': 0.8936192711102957,
 'cross_entropy': 0.5202054542218015,
 'precision': 0.8316831683168316,
 'recall_score': 0.875,
 'npv': 0.6923076923076923,
 'accuracy_score': 0.7928571428571428,
 'bal_accuracy': 0.7443181818181819,
 'kappa': 0.504152418172936,
 'matthews_cc': 0.5060049295355293,
 'confusion_matrix': [[[27, 17], [12, 84]]],
 'num_compounds': 140}